# V Rishikesh

# Task-2(ADVANCED LEVEL TASK)

# Next Word Prediction

Importing Libraries

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

Loading the Dataset

In [2]:
path = '1661-0.txt'
text = open(path).read().lower()
print('Corpus Length:', len(text))

Corpus Length: 581887


# Splitting Dataset into one word each

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

Making list of Sorted Unique Words

In [4]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

# Feature Engineering

In [5]:
WORD_LENGTH = 4
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the']
adventures


Storing Features and Corresponding Labels

In [6]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [7]:
print(X[0][0])

[False False False ... False False False]


# Building the Model

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

Training Our Model

In [9]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
811/811 [==============================] - 147s 178ms/step - loss: 6.0084 - accuracy: 0.1081 - val_loss: 7.1220 - val_accuracy: 0.1022
Epoch 2/20
811/811 [==============================] - 141s 173ms/step - loss: 5.7568 - accuracy: 0.1475 - val_loss: 7.8928 - val_accuracy: 0.1075
Epoch 3/20
811/811 [==============================] - 135s 166ms/step - loss: 5.7435 - accuracy: 0.1767 - val_loss: 8.1226 - val_accuracy: 0.1071
Epoch 4/20
811/811 [==============================] - 138s 171ms/step - loss: 5.4487 - accuracy: 0.2102 - val_loss: 7.9059 - val_accuracy: 0.0998
Epoch 5/20
811/811 [==============================] - 273s 337ms/step - loss: 5.1534 - accuracy: 0.2505 - val_loss: 8.2882 - val_accuracy: 0.1060
Epoch 6/20
811/811 [==============================] - 172s 212ms/step - loss: 4.9112 - accuracy: 0.2856 - val_loss: 8.6047 - val_accuracy: 0.0974
Epoch 7/20
811/811 [==============================] - 143s 177ms/step - loss: 4.7177 - accuracy: 0.3230 - val_loss: 8.6567 -

Loading Our Model

In [10]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

Making Predictions

In [11]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [12]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

Function for Prediction

In [13]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [14]:
s =  "Will you be my partner?"
print("Sentence: ",s)
seq = " ".join(tokenizer.tokenize(s.lower())[0:2])
print("Sequence: ",seq)
print("Next Possible Words: ", predict_completions(seq, 2))

Sentence:  Will you be my partner?
Sequence:  will you
will
you
Next Possible Words:  ['me', 'said']


In [15]:
sen =  "This is not going anywhere."
print("Sentence: ",sen)
sequ = " ".join(tokenizer.tokenize(sen.lower())[0:4])
print("Sequence: ",sequ)
print("Next Possible Words: ", predict_completions(sequ, 4))

Sentence:  This is not going anywhere.
Sequence:  this is not going
this
is
not
going
Next Possible Words:  ['on', 'the', 'through', 'all']
